In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))


In [ ]:
X_train = pd.read_csv('../input/train.csv')
print(X_train.shape)
X_train.head()

In [ ]:
import seaborn as sns
y_train = X_train['label']
print(y_train.shape)
sns.countplot(y_train, palette='icefire')

In [ ]:
X_train = X_train.drop(labels='label', axis=1)
print(X_train.shape)
X_train.head()

In [ ]:
select_num = np.random.randint(1000)
example = X_train.iloc[select_num].values.reshape(28, 28)
plt.imshow(example)
plt.title("label: {}".format(y_train.iloc[select_num]), fontsize=23)
plt.axis('off')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

In [ ]:
from sklearn.model_selection import train_test_split
X_train_np = X_train.values / 255 #Normalization
y_train_np = y_train.values
X_train, X_test, y_train, y_test = train_test_split(X_train_np, y_train_np, test_size=0.2, random_state=28)
#Create tensor for data set.
X_Train = torch.from_numpy(X_train).type(torch.double)
y_Train = torch.from_numpy(y_train).type(torch.LongTensor)
X_Test = torch.from_numpy(X_test).type(torch.double)
y_Test = torch.from_numpy(y_test).type(torch.LongTensor)

In [ ]:
print('The shape of X_Train: {}'.format(X_Train.shape))
print('The shape of y_Train: {}'.format(y_Train.shape))
print('The shape of X_Test: {}'.format(X_Test.shape))
print('The shape of y_Test: {}'.format(y_Test.shape))

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        #convolution layer1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        #1 * 28 * 28 -> 16 * 24 * 24
        
        #max pooling 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        #16 * 24 * 24 -> 16 * 12 * 12
        
        #convolution layer2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        #16 * 12 * 12 -> 32 * 8 * 8
        
        #max pooling 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        #32 * 8 * 8 -> 32 * 4 * 4
        
        #full connected layer 1
        self.fc1 = nn.Linear(32 * 4 * 4, 10)
    
    def forward(self, x):
        #convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        #max pooling 1
        out = self.maxpool1(out)
        
        #convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        #max pooling 2
        out = self.maxpool2(out)
        #flatten
        out = out.view(out.size(0), -1)
        
        #full connected layer
        out = self.fc1(out)
        return out


In [ ]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)
#Pytorch train and test sets
train = torch.utils.data.TensorDataset(X_Train, y_Train)
test = torch.utils.data.TensorDataset(X_Test, y_Test)
#data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle=False)

model = CNNModel()

error = nn.CrossEntropyLoss()

#SGD Optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
#model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        #Clear gradients
        optimizer.zero_grad()
        
        
        #Forward propagation
        model = model.double()
        outputs = model(train.double())
        
        #Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        #Calculating gradients
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        count += 1
        if count % 50 == 0:
            #calculate accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                test = Variable(images.view(100, 1, 28, 28))
                #forward propagation
                outputs = model(test)
                #get prediction from the model
                prediction = torch.max(outputs.data, 1)[1]
                
                total += len(labels)
                
                correct += (prediction == labels).sum()
                
            accuracy = 100 * float(correct) / float(total)
            loss_list.append(loss.data)
            accuracy_list.append(accuracy)
            iteration_list.append(count)
            if count % 500 == 0:
                print('Iteration: {}%, loss: {}, accuracy: {}'.format(count, loss.data.item(), accuracy))

In [ ]:
plt.plot(iteration_list, accuracy_list)
plt.xlabel('Number of iteration')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(iteration_list, loss_list)
plt.xlabel('Number of iteration')
plt.ylabel('Loss')
plt.show()

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission.head()

In [ ]:
test_set = pd.read_csv('../input/test.csv')

print(test_set.shape)
test_set.head()

In [ ]:
test_set = pd.read_csv('../input/test.csv')
test_set.head()
test_set_X = test_set.values / 255
test_set_X_tensor = torch.from_numpy(test_set_X)

test = torch.utils.data.TensorDataset(test_set_X_tensor)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
pred_np = np.array([])
for images in test_loader:
    images = images[0]
    test_images = Variable(images.view(100, 1, 28, 28))
    outputs = model(test_images)
    #get prediction from the model
    prediction = torch.max(outputs.data, 1)[1]
    pred_np = np.hstack((pred_np, np.copy(prediction.numpy())))
pred_np.shape

In [ ]:
prediction = pd.DataFrame({"ImageId": np.arange(1, 28001), "Label": pred_np}, dtype=int)
prediction.to_csv('my_submission.csv', index=False)